In [28]:
import pandas as pd
import altair as alt
from urllib.error import HTTPError

In [29]:
def claim(claim_hash, cols_claim, data_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(data_claim[claim_hash],columns=cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(data_claim[claim_hash],columns=cols_claim[claim_hash])
    return df_claim

In [30]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [31]:
user_stats = '1'
hourly_new_users = '2'
wallet_age = '3'
hourly_stats = '4'
users_balance = '5'

In [32]:
cols_claim = {
    user_stats : ['SENDER', 'DURATION', 'AMOUNT'],
    hourly_stats : ['HR', 
                        'DEP_AMOUNT', 'DEPOSIT_TX', 'DEP_USERS',
                        'WITH_AMOUNT', 'WITH_TX', 'WITH_USERS',
                        'DEP_AMOUNT_3', 'DEP_AMOUNT_6', 'DEP_AMOUNT_9','DEP_AMOUNT_12', 'DEP_AMOUNT_15', 'DEP_AMOUNT_18',
                        'WITH_AMOUNT_3', 'WITH_AMOUNT_6', 'WITH_AMOUNT_9','WITH_AMOUNT_12', 'WITH_AMOUNT_15', 'WITH_AMOUNT_18',
    ],
    wallet_age : ['ADDRESS_COUNT', 'MIN_DATE'],
    hourly_new_users: ['TIME','NEW_USERS'],
    users_balance: ['SENDER','BALANCE'],
}

In [33]:
data_claim = {
    user_stats : [['user1_1',3,10],
                  ['user1_2',3,3],
                  ['user1_3',3,4],
                  ['user1_4',3,67],
                  ['user1_5',3,33],
                 ['user1',9,20],
                 ['user1',18,15],
                 ['user2',3,10],
                 ['user2',6,120],
                 ['user2',18,13],
                 ['user2',3,10],
                 ['user3',6,120],
                 ['user3',18,13]],
    hourly_stats : [['2021-09-21T07:00:00Z',1000,4,3,
                    100,1,1,
                    10,20,30,40,50,60,
                    0,0,10,0,0,0],
                   ['2021-09-21T08:00:00Z',1000,4,3,
                    200,1,1,
                    10,10,30,10,50,5,
                    0,0,0,0,7,0],
                   ['2021-09-21T09:00:00Z',1000,4,3,
                    200,1,1,
                    10,10,30,10,50,5,
                    0,0,0,0,7,0]],
    wallet_age : ['ADDRESS_COUNT', 'MIN_DATE'],
    hourly_new_users: [['2021-09-21T07:00:00Z',1000],
                   ['2021-09-21T08:00:00Z',600],
                   ['2021-09-21T09:00:00Z',200]],
    users_balance: [['user1',100],
                    ['user1_2',20],
                    ['user1_3',50],
                    ['user1_4',70],
                    ['user1_5',80],
                    ['user2',70],
                    ['user3',80]],
}

In [57]:
aust_balance = pd.read_csv('../data/balances/aUST.csv', index_col=1).drop(columns=['Unnamed: 0'])
bluna_balance = pd.read_csv('../data/balances/bLUNA.csv', index_col=1).drop(columns=['Unnamed: 0'])
aust_price = 1.19
bluna_price = 54
aust_balance = aust_balance /1000000 * aust_price
bluna_balance = bluna_balance /1000000 * bluna_price

In [60]:
balances = aust_balance.reset_index().merge(bluna_balance.reset_index(), on='address', how='outer').fillna(0)
balances[balances.address=='terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv']

,address,aUST_balance,bLuna_balance
61869,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,530.179123,22.302108


In [7]:
cols_dict = {}

In [8]:
with_phase1 = get_url('https://raw.githubusercontent.com/IncioMan/prism_forge/master/data/with_phase1.csv')
with_phase1

,sender,deposit,total_deposit,withdrawable_amount,tokens_to_claim,can_claim
0,terra12ca3npchsxgsmxuzwqr9vy257zh4ej9t9emjj3,960.000000,2.364414e+07,0.0,2842.141433,False
1,terra19f9jgp4y0aslj2tf4lhwptrq6jm96s2sqjvutq,0.000000,2.364414e+07,0.0,0.000000,False
2,terra1g9twcftmkxzn005judkrpez4h9eusvpenyvdjg,1000.000000,2.364414e+07,0.0,2960.563993,False
3,terra1sum9e6d6vxsg0yq62axn3hm8nptvwyy0dw8e08,70.000000,2.364414e+07,0.0,207.239479,False
4,terra14jvzcpn3wr57k572pepg3a0f862v8969m2d5kh,500.000000,2.364414e+07,0.0,1480.281996,False
...,...,...,...,...,...,...
5468,terra1ssfpxgkzt4yj0unmzx0cmrx6mhm5kl0g2mvy3k,76280.708184,2.364414e+07,0.0,225833.918028,False
5469,terra100dxspkurxsa29upl7s9pywqxtff70r6l9ux2u,1250.000000,2.364414e+07,0.0,3700.704991,False
5470,terra1xeve8vkkltsd9uzaqya78ywtspyc35hrvsvkvt,400.000000,2.364414e+07,0.0,1184.225597,False
5471,terra1wvlz4xfdz6yzq39accyuxkel4qp30lf38lmfsm,120.540000,2.364414e+07,0.0,356.866383,False


### Data

**User stats**

In [9]:
user_stats_df = claim(user_stats,cols_claim,data_claim)
user_stats_df.columns = [c.lower() for c in user_stats_df.columns]
user_stats_df

,sender,duration,amount
0,user1_1,3,10
1,user1_2,3,3
2,user1_3,3,4
3,user1_4,3,67
4,user1_5,3,33
5,user1,9,20
6,user1,18,15
7,user2,3,10
8,user2,6,120
9,user2,18,13


In [10]:
count_durations_users = \
    user_stats_df.groupby('sender').duration.nunique().reset_index()\
                 .groupby('duration').sender.count().reset_index()\
                 .rename(columns={
                            'duration':'Number of lockup durations',
                            'sender':'Number of users'})
count_durations_users

,Number of lockup durations,Number of users
0,1,5
1,2,2
2,3,1


**Hourly stats**

In [11]:
hourly_stats_df = claim(hourly_stats,cols_claim,data_claim)
hourly_stats_df.columns = [c.lower() for c in hourly_stats_df.columns]
hourly_stats_df

,hr,dep_amount,deposit_tx,dep_users,with_amount,with_tx,with_users,dep_amount_3,dep_amount_6,dep_amount_9,dep_amount_12,dep_amount_15,dep_amount_18,with_amount_3,with_amount_6,with_amount_9,with_amount_12,with_amount_15,with_amount_18
0,2021-09-21T07:00:00Z,1000,4,3,100,1,1,10,20,30,40,50,60,0,0,10,0,0,0
1,2021-09-21T08:00:00Z,1000,4,3,200,1,1,10,10,30,10,50,5,0,0,0,0,7,0
2,2021-09-21T09:00:00Z,1000,4,3,200,1,1,10,10,30,10,50,5,0,0,0,0,7,0


In [12]:
hourly_stats_df['tot_txs'] = hourly_stats_df.with_tx + hourly_stats_df.deposit_tx
hourly_stats_df['net_deposit_3'] = (hourly_stats_df.dep_amount_3-hourly_stats_df.with_amount_3).cumsum()
hourly_stats_df['net_deposit_6'] = (hourly_stats_df.dep_amount_6-hourly_stats_df.with_amount_6).cumsum()
hourly_stats_df['net_deposit_9'] = (hourly_stats_df.dep_amount_9-hourly_stats_df.with_amount_9).cumsum()
hourly_stats_df['net_deposit_12'] = (hourly_stats_df.dep_amount_12-hourly_stats_df.with_amount_12).cumsum()
hourly_stats_df['net_deposit_15'] = (hourly_stats_df.dep_amount_15-hourly_stats_df.with_amount_15).cumsum()
hourly_stats_df['net_deposit_18'] = (hourly_stats_df.dep_amount_18-hourly_stats_df.with_amount_18).cumsum()
hourly_stats_df['tot_txs'] = hourly_stats_df.with_tx + hourly_stats_df.deposit_tx


In [13]:
time_duration_df = hourly_stats_df[['hr','net_deposit_3','net_deposit_6','net_deposit_9','net_deposit_12','net_deposit_15','net_deposit_18']]
time_duration_df = time_duration_df.rename(columns={
    'net_deposit_3':'3 months',
    'net_deposit_6':'6 months',
    'net_deposit_9':'9 months',
    'net_deposit_12':'12 months',
    'net_deposit_15':'15 months',
    'net_deposit_18':'18 months'
})
time_duration_df = time_duration_df.melt(id_vars=["hr"], 
        var_name="Lockup period", 
        value_name="UST deposited")


**New users hourly**

In [14]:
hourly_new_users_df = claim(hourly_new_users,cols_claim,data_claim)
hourly_new_users_df.columns = [c.lower() for c in hourly_new_users_df.columns]
hourly_new_users_df=hourly_new_users_df.sort_values(by='time',ascending=True)
hourly_new_users_df['cumsum_new_users'] = hourly_new_users_df.new_users.cumsum()
hourly_new_users_df


,time,new_users,cumsum_new_users
0,2021-09-21T07:00:00Z,1000,1000
1,2021-09-21T08:00:00Z,600,1600
2,2021-09-21T09:00:00Z,200,1800


In [15]:
last_duration_amount = hourly_stats_df[hourly_stats_df.hr==hourly_stats_df.hr.max()]
last_duration_amount= last_duration_amount[['net_deposit_3','net_deposit_6','net_deposit_9',
                     'net_deposit_12','net_deposit_15','net_deposit_18']]
last_duration_amount = last_duration_amount.rename(columns={
            'net_deposit_3':'3 months',
            'net_deposit_6':'6 months',
            'net_deposit_9':'9 months',
            'net_deposit_12':'12 months',
            'net_deposit_15':'15 months',
            'net_deposit_18':'18 months'
        })
last_duration_amount = last_duration_amount.T.reset_index()
last_duration_amount.columns = ['Lockup period','UST deposited']
last_duration_amount


,Lockup period,UST deposited
0,3 months,30
1,6 months,40
2,9 months,80
3,12 months,60
4,15 months,136
5,18 months,70


**User balance**

In [16]:
users_balance_df = claim(users_balance,cols_claim,data_claim)
users_balance_df.columns = [c.lower() for c in users_balance_df.columns]
users_balance_df

,sender,balance
0,user1,100
1,user1_2,20
2,user1_3,50
3,user1_4,70
4,user1_5,80
5,user2,70
6,user3,80


In [17]:
user_stats_df['dur_amount']=user_stats_df.duration * user_stats_df.amount

In [18]:
df = user_stats_df.groupby('sender').agg(mean_duration=('duration', 'mean'),
                                              dur_sum=('dur_amount', 'sum'),
                                              amnt_sum=('amount', 'sum'))
df['weighted_avg_dur'] = df.dur_sum/df.amnt_sum

In [19]:
users_balance_df = df.reset_index().merge(users_balance_df, on='sender')
users_balance_df

,sender,mean_duration,dur_sum,amnt_sum,weighted_avg_dur,balance
0,user1,13.5,450,35,12.857143,100
1,user1_2,3.0,9,3,3.000000,20
2,user1_3,3.0,12,4,3.000000,50
3,user1_4,3.0,201,67,3.000000,70
4,user1_5,3.0,99,33,3.000000,80
5,user2,7.5,1014,153,6.627451,70
6,user3,12.0,954,133,7.172932,80


### Charts

In [20]:
users_balance_df['url'] = 'https://finder.extraterrestrial.money/mainnet/address/'+users_balance_df['sender']
if(len(users_balance_df)>5000):
    df = users_balance_df.sample(n=5000, random_state=1)
else:
    df = users_balance_df
dep_dist_balance_chart =alt.Chart(df).mark_point(opacity=1, filled=True).encode(
y=alt.Y("amnt_sum:Q",scale=alt.Scale(domain=(0, 200))),
x=alt.X("balance:Q",scale=alt.Scale(domain=(0, 200))),
href='url:N',
color=alt.Color('weighted_avg_dur',
    scale=alt.Scale(scheme='redpurple')),
tooltip=['sender', 'amnt_sum','balance','weighted_avg_dur']
).configure_view(strokeOpacity=0).interactive()
dep_dist_balance_chart

alt.Chart(...)

In [27]:
pie_ust_chart = alt.Chart(last_duration_amount).mark_bar().encode(
    y=alt.Y(field="UST deposited", type="quantitative"),
    x=alt.X(field="Lockup period", type="nominal"),
    color=alt.Color("Lockup period",
            sort=['3 months','6 months',
                              '9 months','12 months',
                              '15 months','18 months'],
            scale=alt.Scale(scheme='pastel1'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-40,
            direction='vertical')),
    tooltip=["UST deposited","Lockup period"]
).configure_view(strokeOpacity=0)
pie_ust_chart

alt.Chart(...)

In [22]:
df = hourly_new_users_df.rename(columns={'time':'Time','cumsum_new_users':'Number of total users'})
users_over_time_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Time:T',\
        axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
    y="Number of total users:Q",
    tooltip=['Time', "Number of total users:Q"]
    ).configure_mark(
        color='#fab0ba'
    ).configure_view(strokeOpacity=0)
users_over_time_chart

alt.Chart(...)

In [164]:
df = hourly_stats_df.rename(columns={'hr':'Time','tot_txs':'Number of transactions'})
txs_over_time_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Time:T', \
            axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
    y="Number of transactions:Q",
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),"Number of transactions:Q"]
).configure_mark(
    color='#ffde85'
).configure_view(strokeOpacity=0)
txs_over_time_chart

alt.Chart(...)

In [152]:
alt.Chart(time_duration_df.rename(columns={'hr':'Time'})).mark_line(point = True).encode(
    x='Time:T',
    y='UST deposited:Q',
    color=alt.Color('Lockup period:N', scale=alt.Scale(scheme='pastel1')),
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),'UST deposited:Q','Lockup period:N']
)

alt.Chart(...)

In [88]:
n_tx_wallet_chart = alt.Chart(count_durations_users).mark_line(point = True, color='#7DDBD3').encode(
    y=alt.Y('Number of users:Q', sort="ascending"),
    x="Number of lockup durations:O",
    tooltip=['Number of users:Q',"Number of lockup durations:Q"]
).properties(height=300).configure_view(strokeOpacity=0)
n_tx_wallet_chart

alt.Chart(...)